In [4]:
import os, json
from datetime import datetime, timezone
from pymongo import MongoClient, ReturnDocument
from dotenv import load_dotenv
from bson.decimal128 import Decimal128

# ---------------------------
# Config & Connect
# ---------------------------
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

if not uri or not database_name:
    raise RuntimeError("Missing MONGODB_URI or MONGODB_DATABASE.")

client = MongoClient(uri)
db = client[database_name]
col = db[collection_name]

# ---------------------------
# Updated payload (example)
# ---------------------------
payload = {
    "id": "2b877c11-acbc-40f4-80c5-9bd13bb2d71f",
    "current_form_qr_code": "20251013",
    "previous_form_qr_code": "20251012",
    "tin": "123-456-789",
    "location": "JEF Gas Station – Sikatuna Branch",
    "created": "2025-10-13T05:03:46.557977Z",
    "date": "2025-10-13",
    "items": [
        {
            "receipt_number": "123",
            "customer_name": "Markus",
            "type": "fuel",
            "vatable_sales": 1111,
            "vat_amount": 133.32,
            "total_amount": 1244.32
        },
        {
            "receipt_number": "222",
            "customer_name": "Just",
            "type": "fuel",
            "vatable_sales": 22222,
            "vat_amount": 2666.64,
            "total_amount": 24888.64
        }
    ],
    "cashier_employee_number": "10001",
    "cashier": "John Smith",
    "recorder_employee_number": "10003",
    "recorder": "David Williams"
}

# ---------------------------
# Helpers
# ---------------------------
REQUIRED_TOP_LEVEL = [
    "id", "current_form_qr_code", "previous_form_qr_code",
    "tin", "location", "date", "items",
    "cashier_employee_number", "cashier",
    "recorder_employee_number", "recorder"
]

def _to_decimal128(x):
    if isinstance(x, Decimal128):
        return x
    return Decimal128(str(x))

def _normalize_payload(doc):
    d = dict(doc)
    items = []
    for it in d.get("items", []):
        it = dict(it)
        for k in ("vatable_sales", "vat_amount", "total_amount"):
            if k in it and it[k] is not None:
                it[k] = _to_decimal128(it[k])
        items.append(it)
    d["items"] = items
    return d

def _validate_payload(d):
    missing = [k for k in REQUIRED_TOP_LEVEL if k not in d]
    if missing:
        raise RuntimeError(f"❌ Payload missing required fields: {missing}")
    if not isinstance(d.get("items"), list) or len(d["items"]) == 0:
        raise RuntimeError("❌ Payload.items must be a non-empty list.")

_validate_payload(payload)
norm = _normalize_payload(payload)

# ---------------------------
# Ensure there is EXACTLY ONE is_empty=True, then patch it and set is_empty=False
# ---------------------------
count_empty = col.count_documents({"is_empty": True})

if count_empty == 0:
    raise RuntimeError("❌ No document with is_empty=True found.")
if count_empty > 1:
    raise RuntimeError(f"❌ Expected exactly one is_empty=True document, found {count_empty}.")

only_empty = col.find_one({"is_empty": True})
if not only_empty:
    raise RuntimeError("❌ Race condition: is_empty=True doc disappeared.")

update_op = {
    "$set": {
        "id": norm["id"],
        "current_form_qr_code": norm["current_form_qr_code"],
        "previous_form_qr_code": norm["previous_form_qr_code"],
        "tin": norm["tin"],
        "location": norm["location"],
        "date": norm["date"],
        "cashier_employee_number": norm["cashier_employee_number"],
        "cashier": norm["cashier"],
        "recorder_employee_number": norm["recorder_employee_number"],
        "recorder": norm["recorder"],
        "items": norm["items"],
        "updated_at": datetime.now(timezone.utc).isoformat(),
        "is_empty": False
    }
}

patched = col.find_one_and_update(
    filter={"_id": only_empty["_id"], "is_empty": True},
    update=update_op,
    return_document=ReturnDocument.AFTER
)

if patched is None:
    raise RuntimeError("❌ Patch failed — the target doc was modified by someone else.")

print("✅ Patched the only is_empty=True document and set is_empty=False.")
print(json.dumps(patched, indent=2, default=str))


RuntimeError: ❌ No document with is_empty=True found.